# ECDSA - *Elliptic Curve Digital Signature Algorithm*

Segue-se a implementação do ECDSA, usando a curva elitica prima **P-521**, definida no **FIPS186-4**. Note-se que, para a mesma, tomou-se como referência as páginas  394 e 395 do seguinte livro: *S. Yan, Number Theory for Computing, Springer, 2002*.

Seja **E** uma curva elíptica sobre $\mathbb{F}_{p}$, sendo $p$ primo. Seja ainda P um ponto da curva de ordem prima $q$ em $E(\mathbb{F}_{p})$.

## SETUP

O estabelecimento dos parâmetros usados no processo de assinatura e verificação pode ser descrito da seguinte maneira:

1. Gerar aleatoriamente um inteiro $x \in [1,q-1]$ que será a **chave privada**

2. Calcular $Q = xP$ que será a **chave pública**

## GERAÇÃO DA ASSINATURA

O processo de assinatura de uma mensagem $m$ segue os seguintes passos:

1. Gerar aleatoriamente um inteiro $k \in [1,q-1]$;

2. Calcular $kP = (x_{1},y_{1})$ tal que $r\equiv x_{1} (mod\ q)$. Se $r=0$, voltar ao ponto 1;

3. Calcular $k^{-1} mod\ q$;

4. Calcular $s \equiv k^{-1}(sha256(m)+x*r)(mod q)$. Se $s=0$, voltar ao ponto 1.

A assinatura da mensagem $m$ é o par de inteiros $(r,s).$

## VERIFICAÇÃO DA ASSINATURA

Dados: 
- uma assinatura $(r,s)$ 
- uma mensagem $m$
- a chave pública $Q$ associada à chave privada usada para assinar a mensagem

o processo de verificação pode ser descrito da seguinte maneira:

1. Determinar $w \equiv s^{-1}(mod\ q)$ e $h(m)$;

2. Calcular $u_{1}\equiv sha256(m)\ w\  (mod\ q)$ e $u_{2}\equiv r\ w\ (mod\ q)$;

3. Determinar $u_{1}P + u_{2}Q = (x_0,y_0)$ e $v \equiv x_{0}(mod q)$;

4. A assinatura é validade se e só se: $v = r$.

In [16]:
import hashlib

class ECDSA:
    def __init__(self):
        p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
        self.q = 6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
        b = Integer(0x051953eb9618e1c9a1f929a21a0b68540eea2da725b99b315f3b8b489918ef109e156193951ec7e937b1652c0bd3bb1bf073573df883d2c34f1ef451fd46b503f00)
        E = EllipticCurve(GF(p),[-3, b])
        E.set_order(self.q)
        self.P = E.random_point()
        self.priv_key = ZZ.random_element(1, self.q)
        self.pub_key = self.priv_key * self.P

    def sign(self, m):
        sha256 = hashlib.sha256()
        sha256.update(m)
        m_hash = int(sha256.hexdigest(), 16)
        r = s = 0
        while not(r and s):
            k = ZZ.random_element(1,self.q)
            x = (k*self.P)[0]
            r = mod(x, self.q)
            if r:
                k_1 = mod(k^(-1), self.q)
                s = mod((k_1 * (m_hash + self.priv_key * r)), self.q)
        sig = (r,s)
        return sig

    def verify(self, sig ,m):
        r = sig[0]
        s = sig[1]
        w = mod(s^(-1),self.q)
        m_hash = int(hashlib.sha256(m).hexdigest(), 16)
        u_1 = ZZ(mod(m_hash*w, self.q))
        u_2 = ZZ(mod(r*w, self.q))
        x_0 = (u_1*self.P+u_2*self.pub_key)[0]
        v = mod(x_0, self.q)
        return (v == r)

In [17]:
ecdsa = ECDSA()
msg = "Mensagem";
assinatura = ecdsa.sign(msg)

In [18]:
ecdsa.verify(assinatura,msg)

True